In [ ]:
### How they do it in the Readme file ###
import numpy as np
import h5py

def load_data(hdf5_path):
    with h5py.File(hdf5_path, 'r') as hf:
        x = hf.get('x')
        y = hf.get('y')
        video_id_list = hf.get('video_id_list')
        x = np.array(x)
        y = list(y)
        video_id_list = list(video_id_list)
        
    return x, y, video_id_list

def uint8_to_float32(x):
    return (np.float32(x) - 128.) / 128.
    
def bool_to_float32(y):
    return np.float32(y)

path = '../packed_features/' #just if they can be found in other folders
files = ['bal_train','eval'] #forget about the 'unbal_train'S
extension = '.h5'
data = {}
for f in files:
    
    data[f] = {}
    hdf5_path = path+f+extension
    print('\nReading data from: '+hdf5_path)
    
    (x, y, video_id_list) = load_data(hdf5_path)
    x = uint8_to_float32(x)		# shape: (N, 10, 128)
    y = bool_to_float32(y)		# shape: (N, 527)
    data[f]['x'] = np.array(x)
    data[f]['y'] = np.array(y)
    data[f]['video_id_list'] = [str(v)[2:-1] for v in video_id_list]
    
    # print info
    print('x',x.shape)
    print('y',y.shape)
    print('video id',len(video_id_list))
    

In [ ]:
# Let's have a look at some of the data
for d in data.keys():
    print('\n\n'+d+':')
    params = ['x','y','video_id_list']
    for p in params:
        print('\n'+p+':')
        print(data[d][p][0])

In [ ]:
v = data[list(data.keys())[1]][params[2]]
find_id = '9OqtuFGCCR8'#'Sb0169-lqLs'
#print(v.)
for i in v:
    if i == find_id:
        print(i,' == '+find_id)
        break

In [ ]:
import os
def get_dir_files(path, just_filename=True):
    files = []
    for (dirpath, dirnames, filenames) in os.walk(path):
        if just_filename:
            files.extend([file for file in filenames])
        else:
            direc = dirpath.split('/')[-1]
            files.extend([direc+'/'+file for file in filenames])
    print('get_dir_files\nFiles: ',files)
    return files

audioset_path = '../packed_features/'
audioset_files = get_dir_files(path=audioset_path, just_filename=True)
audioset_files = [f for f in audioset_files if (f.split('.')[-1] == 'h5') and (f.split('_')[0] != 'unbal')]
print(audioset_files)

In [ ]:
import pandas as pd
h = pd.DataFrame()
h['1']= [1,2,3]
h['2']= [4,5,6]
h['id']= ['a','b','c']
h.set_index('id',inplace=True)
print(h.head())

j = pd.DataFrame()
j['4']= [41,42,43]
j['3']= [34,35,36]
j['id']= ['a','b','d']
j.set_index('id',inplace=True)
print(j.head())

result = pd.merge(h, j, on='id', how='inner')
print(result.head())

In [ ]:
q = np.array([[11,12,13], [31,32,33]])
p = np.array([[21,22], [41,42]])
print(q)
print(p)
print(np.concatenate((q.T, p.T), axis=0))
print(np.concatenate((np.array([]),p), axis=None))
a = pd.DataFrame()
a['a'] = np.concatenate((q.T, p.T), axis=0)
print(a.head())

In [ ]:
def read_h5_files(path, files, print_info=False):
    #initialization of outputs
    data = {}
    pd_data = pd.DataFrame()
    x_pd = np.array([])
    y_pd = np.array([])
    id_pd = np.array([])

    for f in files:
        data[f] = {}
        hdf5_path = path+f
        print('\nReading data from: '+hdf5_path)
        
        (x, y, video_id_list) = load_data(hdf5_path)
        x = np.array(uint8_to_float32(x))     # shape: (N, 10, 128)
        y = np.array(bool_to_float32(y))      # shape: (N, 527)
        video_id_list = np.array([str(v)[2:-1] for v in video_id_list]) # take just the ID

        data[f]['x'] = x
        data[f]['y'] = y
        data[f]['id'] = video_id_list

        #pd_data['x']
        x_pd= np.concatenate((x_pd,x), axis=0)
        y_pd= np.concatenate((y_pd,y), axis=0)
        id_pd= np.concatenate((id_pd,id), axis=0)
        #pd_data['y'] = np.concatenate((pd_data['y'],y), axis=None)
        #pd_data['id'] = np.concatenate((pd_data['id'],video_id_list), axis=None)
        
        # print info
        if print_info:
            print('x',x.shape)
            print('y',y.shape)
            print('video id',len(video_id_list))
    
        print(x_pd.shape)
        print(y_pd.shape)
        print(id_pd.shape)
        
    """
    pd_data['x'] = x_pd
    pd_data['y'] = y_pd
    pd_data['id'] = id_pd
    pd_data.set_index('id',inplace=True)
    """
    return {'dict_data':data, 'pd_data':pd_data}
    
audioset_path = '../packed_features/'
audioset_files = get_dir_files(path=audioset_path, just_filename=True)
# Keep just the ones with extension .h5 and NOT unbalanced data!
audioset_files = [f for f in audioset_files if (f.split('.')[-1] == 'h5') and (f.split('_')[0] != 'unbal')]
print(audioset_files)
audioset_data = read_h5_files(path=audioset_path, files=audioset_files, print_info=True)
# because we want it in DataFrame and not as dict, see the function
audioset_data = audioset_data['pd_data']
audioset_data.head()

In [ ]:
audioset_data.shape